In [11]:
#1
import numpy as np
from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd

df = pd.read_csv("ZIONO_weekly_return_volatility.csv")
df.label = df.label.fillna(method='ffill')
df_21 = df[df.Year==2021]
df_22 = df[df.Year==2022]

df_22.head(10)


df_22.head() # mean


,Year,Week_Number,mean_return,volatility,label
54,2022,1,-0.2012,0.842329,red
55,2022,2,0.1002,0.604448,green
56,2022,3,0.1525,0.591757,green
57,2022,4,-0.3576,1.847664,red
58,2022,5,-0.1206,0.762250,red


In [12]:
# 1 , construct df_train
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from scipy import stats

x_21 = df_21[["mean_return"]].values
le = LabelEncoder()
y_21 = le.fit_transform(df_21["label"].values)
y_22 = le.transform(df_22["label"].values)


df_train = pd.DataFrame({'x': df_21["mean_return"].values, 'y': y_21})
df_train.head()



,x,y
0,-0.2586,0
1,0.0776,1
2,0.9710,0
3,-0.5042,1
4,0.1458,0


In [13]:
from scipy.stats import norm
# assume data is in array x
df_train_red = df_train[df_train.y == 1]
print(df_train_red.shape)
df_train_red.mean(),df_train_red.std()
red_mu = df_train_red.mean()[0]
red_sigma = df_train_red.std()[0]
red_prop = 30 / 52


# from the below result we can see: 
# for red:
# count = 30
# u = mean = -0.199
# sigma = std = 0.239976
red_mu, red_sigma, red_prop


(30, 2)


(-0.19924333333333333, 0.23997625303973, 0.5769230769230769)

In [14]:
# assume data is in array x
df_train_green = df_train[df_train.y == 0]
print(df_train_green.shape)
df_train_green.mean(),df_train_green.std()

# from the below result we can see: 
# for red:
# count = 22
# u = mean = 0.32
# sigma = std = 0.399191
green_mu = df_train_green.mean()[0]
green_sigma = df_train_green.std()[0]
green_prop = 22 / 52

def get_red_or_green(z):
#     print(z)
    red_prob = norm.pdf((z - red_mu) / red_sigma)
    green_prob = norm.pdf((z - green_mu) / green_sigma)
    p_red = red_prop * 30 / 52
    p_green = green_prop * 22 / 52
    normalized_red = p_red / (p_red + p_green)
    normalized_green = p_green / (p_red + p_green)
    return 1 if normalized_red > normalized_green else 0

green_mu, green_sigma, green_prop

(22, 2)


(0.3204181818181818, 0.39919129545615767, 0.4230769230769231)

In [15]:
# use 21 mean, sigma to compute 2022


# df_22['predict'] = df_22.mean_return.apply(get_red_or_green)
NB_classifier = GaussianNB ().fit( df_21[["mean_return"]].values , y_21 )
df_22['predict'] = NB_classifier.predict(df_22[["mean_return"]].values)
y_22 = le.transform(df_22["label"].values)
ret_df = pd.DataFrame({"pred": df_22['predict'], "ground_label": y_22})
acc = accuracy_score(y_22, df_22['predict'])
print(acc)
ret_df.head()


0.75


d:\ProgramData\Anaconda3\envs\sklearn\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,pred,ground_label
54,1,1
55,1,0
56,0,0
57,1,1
58,1,1


In [16]:
# 2
confusion_matrix(y_22, df_22['predict'])

array([[15, 12],
       [ 1, 24]], dtype=int64)

In [17]:
#3 true negative, true positive, 
confusion_matrix(y_22,  df_22['predict'])[0][0], confusion_matrix(y_22,  df_22['predict'])[1][1]

(15, 24)

In [18]:
tn, fp, fn, tp = confusion_matrix(y_22,  df_22['predict']).ravel()
tn, fp, fn, tp
tnr = tn / (tn + fp)
tpr = tp / (tp + fn)
tnr, tpr

(0.5555555555555556, 0.96)

In [19]:
# 4
# df
df_detail = pd.read_csv("ZIONO_weekly_return_detailed.csv")
df_detail.head(22)


df_detail_22 = df_detail[["Close", "Week_Number", "Year"]]
df_detail_22 = df_detail_22[df_detail_22.Year == 2022]
df_22_week = df_detail_22.groupby("Week_Number").last()
df_22_week.head()



,Close,Year
Week_Number,,
1,26.290001,2022
2,26.420000,2022
3,26.580000,2022
4,26.090000,2022
5,25.930000,2022


In [20]:
df_22_week.tail()

,Close,Year
Week_Number,,
48,25.180000,2022
49,25.129999,2022
50,25.299999,2022
51,25.120001,2022
52,25.370001,2022


In [21]:
# buy_and_hold
buy_hold_add = df_22_week.Close.values[-1] - df_22_week.Close.values[0]
buy_hold_add

-0.9200000762939453

In [22]:
# my strategy
def get_benefit_my_strategy(pred_labels, close_prices):
    money = 100
    hold_share = False
    for i in range(len(pred_labels)): # 1 red
#         print(pred_labels[i], close_prices[i])
        
        if (pred_labels[i] == 1): # red---> if not owning, buy one
            if hold_share:
                pass
            else:
                hold_share = True
                money -= close_prices[i]
        else:           # green ---->if owning, sell it
            if hold_share:
                hold_share = False
                money += close_prices[i]
                
        
    share_value = close_prices[-1] if hold_share else 0
    return money + share_value - 100
                    

get_benefit_my_strategy( df_22['predict'].values.tolist(), df_22_week.Close.values.tolist())

1.519998550415039

In [46]:
# my strategy > buy_and_hold. it is better